In [1]:
from pathlib import Path
import clip
import torch
from PIL import Image
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def compute_clip_features(text_batch):

    # 对所有文本进行预处理
    # Pre-process all texts
    textss=torch.cat([clip.tokenize(f"{i}")for i in text_batch]).to(device)

    with torch.no_grad():
        #对文本提取特征值并标准化
        #Extract features and normalize them
        texts_features = model.encode_text(textss)
        texts_features /= texts_features.norm(dim=-1, keepdim=True)

    #以numpy形式返回特征值
    return texts_features.cpu().numpy()


import math
import numpy as np
import pandas as pd
from pathlib import Path
#按批次进行处理，batch_size可以根据自己机器性能自定义
batch_size = 32

#设置特征值存放目录
# Path where the feature vectors will be stored
features_path = Path("action_data") / "features"

#读取待处理文本 Load texts
f=open("k400cls.txt",mode='r+')
s=open("states.txt",mode='r+')
actions=list()
f_contents=f.readlines()
s_contents=s.readlines()
for i in f_contents:
    for j in s_contents:
        #嵌入动作和状态文本 Embed action and state desicirption
        actions.append(f"human action of {i} {j}")
        
# 计算batch数量 Compute how many batches are needed
batches = math.ceil(len(f_contents)*len(s_contents) / batch_size)


# 处理所有批次 Process each batch
for i in range(batches):
    print(f"Processing batch {i+1}/{batches}")
    batch_ids_path = features_path / f"{i:010d}.csv"
    batch_features_path = features_path / f"{i:010d}.npy"
    
    # 当该批次未处理时处理该批次并保存 Only do the processing if the batch wasn't processed yet
    if not batch_features_path.exists():
        try:
            # Select the photos for the current batch
            batch_files = actions[i*batch_size : (i+1)*batch_size]

            # Compute the features and save to a numpy file
            batch_features = compute_clip_features(batch_files)
            np.save(batch_features_path, batch_features)

            # Save the photo IDs to a CSV file
            text = [i for i in batch_files]
            photo_ids_data = pd.DataFrame(text, columns=['text_content'])
            photo_ids_data.to_csv(batch_ids_path, index=False)
        except:
            # Catch problems with the processing to make the process more robust
            print(f'Problem with batch {i}')
            
# 读取所有处理好的numpy文件 Load all numpy files
features_list = [np.load(features_file) for features_file in sorted(features_path.glob("*.npy"))]

# 合并所有特征文件 Concatenate the features and store in a merged file
features = np.concatenate(features_list)
np.save(features_path / "features.npy", features)

# 合并所有文本文件
photo_ids = pd.concat([pd.read_csv(ids_file) for ids_file in sorted(features_path.glob("*.csv"))])
photo_ids.to_csv(features_path / "text_content.csv", index=False)

Processing batch 1/169
Processing batch 2/169
Processing batch 3/169
Processing batch 4/169
Processing batch 5/169
Processing batch 6/169
Processing batch 7/169
Processing batch 8/169
Processing batch 9/169
Processing batch 10/169
Processing batch 11/169
Processing batch 12/169
Processing batch 13/169
Processing batch 14/169
Processing batch 15/169
Processing batch 16/169
Processing batch 17/169
Processing batch 18/169
Processing batch 19/169
Processing batch 20/169
Processing batch 21/169
Processing batch 22/169
Processing batch 23/169
Processing batch 24/169
Processing batch 25/169
Processing batch 26/169
Processing batch 27/169
Processing batch 28/169
Processing batch 29/169
Processing batch 30/169
Processing batch 31/169
Processing batch 32/169
Processing batch 33/169
Processing batch 34/169
Processing batch 35/169
Processing batch 36/169
Processing batch 37/169
Processing batch 38/169
Processing batch 39/169
Processing batch 40/169
Processing batch 41/169
Processing batch 42/169
P